<a href="https://colab.research.google.com/github/orevs-com/Stock-Market-Price-Prediction/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Market Price Prediction using LSTM : A comaparative Analysis
# by Orevaoghene Otiede

Importing the required libraries for the project


In [9]:
#Importing the Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import linear_model
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
import keras.backend as K
import yfinance as yf
import os

In [10]:
# Import the drive module from the google.colab package.
from google.colab import drive

# Mount the Google Drive to the specified mount point in the Colab environment.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


List of ticker symbols for Training Companies

In [11]:
# Create a list of Ticker Symbols
train_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META']
test_tickers = ['TSLA', 'AVGO', 'COST', 'NFLX', 'ADBE', 'INTC']

Downloading and Saving the Stock Market Price Data

In [13]:
# Define the date range
start_date = "2015-01-01"
end_date = "2024-12-31"
# Define the file path
drive_path = '/content/drive/MyDrive/Colab Notebooks/Stock Market Data/'

print(f"Downloading daily stock data from {start_date} to {end_date} and saving individually...")

# Function to download and save data for a list of tickers
def download_and_save_tickers(tickers, start, end, base_path):
    for ticker in tickers:
        try:
            print(f"Fetching data for {ticker}...")
            # auto_adjust=True is crucial for adjusted close prices
            df = yf.download(ticker, start=start, end=end, interval="1d", auto_adjust=True)

            if not df.empty:
                file_path = os.path.join(base_path, f"{ticker}.csv")
                df.to_csv(file_path)
                print(f"Successfully downloaded and saved data for {ticker} to {file_path}")
            else:
                print(f"No data found for {ticker} in the specified date range.")

        except Exception as e:
            print(f"Error downloading data for {ticker}: {e}")

# Download and save training data
print("\n--- Downloading Training Data ---")
download_and_save_tickers(train_tickers, start_date, end_date, drive_path)

# Download and save testing data
print("\n--- Downloading Testing Data ---")
download_and_save_tickers(test_tickers, start_date, end_date, drive_path)

print("\nAll data download attempts complete. Check the specified Google Drive folder for individual CSVs.")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


--- Downloading Training Data ---
Fetching data for AAPL...
Successfully downloaded and saved data for AAPL to /content/drive/MyDrive/Colab Notebooks/Stock Market Data/AAPL.csv
Fetching data for MSFT...
Successfully downloaded and saved data for MSFT to /content/drive/MyDrive/Colab Notebooks/Stock Market Data/MSFT.csv
Fetching data for GOOGL...



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Successfully downloaded and saved data for GOOGL to /content/drive/MyDrive/Colab Notebooks/Stock Market Data/GOOGL.csv
Fetching data for AMZN...
Successfully downloaded and saved data for AMZN to /content/drive/MyDrive/Colab Notebooks/Stock Market Data/AMZN.csv
Fetching data for NVDA...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

...

Successfully downloaded and saved data for TSLA to /content/drive/MyDrive/Colab Notebooks/Stock Market Data/TSLA.csv
Fetching data for AVGO...
Successfully downloaded and saved data for AVGO to /content/drive/MyDrive/Colab Notebooks/Stock Market Data/AVGO.csv
Fetching data for COST...



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Successfully downloaded and saved data for COST to /content/drive/MyDrive/Colab Notebooks/Stock Market Data/COST.csv
Fetching data for NFLX...
Successfully downloaded and saved data for NFLX to /content/drive/MyDrive/Colab Notebooks/Stock Market Data/NFLX.csv
Fetching data for ADBE...



[*********************100%***********************]  1 of 1 completed

Successfully downloaded and saved data for ADBE to /content/drive/MyDrive/Colab Notebooks/Stock Market Data/ADBE.csv
Fetching data for INTC...
Successfully downloaded and saved data for INTC to /content/drive/MyDrive/Colab Notebooks/Stock Market Data/INTC.csv

All data download attempts complete. Check the specified Google Drive folder for individual CSVs.
